In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_rows', 10)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
print("Vis setup Complete")
print("Setup complete.")

Vis setup Complete
Setup complete.


In [9]:
loan_data_path = "J:\LendingClubLoans2018-2020.csv";
loan_data = pd.read_csv(loan_data_path)
loan_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77159 entries, 0 to 77158
Data columns (total 108 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   id                                          int64  
 1   emp_title                                   object 
 2   application_type                            object 
 3   home_ownership                              object 
 4   loan_status                                 object 
 5   loan_amnt                                   int64  
 6   int_rate                                    object 
 7   term                                        object 
 8   installment                                 float64
 9   grade                                       object 
 10  earliest_cr_line                            object 
 11  issue_d                                     object 
 12  last_pymnt_d                                object 
 13  next_pymnt_d                  

In [58]:
feature_missing_values = loan_data.isna().sum()
feature_missing_values = feature_missing_values.to_frame('Missing_Values')

In [61]:
percenatges_df = (loan_data.isna().sum()* 100 / len(loan_data)).to_frame('Missing_Percentages')
missing_values_stats_df = pd.concat([feature_missing_values, percenatges_df], axis=1)
missing_values_stats_df


,Missing_Values,Missing_Percentages
id,0,0.000000
emp_title,5862,7.597299
application_type,0,0.000000
home_ownership,0,0.000000
loan_status,0,0.000000
...,...,...
deferral_term,72668,94.179551
hardship_amount,71042,92.072215
hardship_payoff_balance_amount,71042,92.072215
hardship_last_payment_amount,71042,92.072215


In [65]:
missing_values_stats_df.loc['deferral_term']

Missing_Values         72668.000000
Missing_Percentages       94.179551
Name: deferral_term, dtype: float64

In [82]:
missing_values_stats_df[missing_values_stats_df['Missing_Percentages'] > 60]

,Missing_Values,Missing_Percentages
annual_inc_joint,67777,87.840693
verification_status_joint,67780,87.844581
hardship_reason,72668,94.179551
hardship_type,72668,94.179551
hardship_status,72668,94.179551
...,...,...
deferral_term,72668,94.179551
hardship_amount,71042,92.072215
hardship_payoff_balance_amount,71042,92.072215
hardship_last_payment_amount,71042,92.072215


In [81]:
more_null_values = []
for index, row in missing_values_stats_df.iterrows():
    if (row['Missing_Percentages'] > 60):
        more_null_values.append(index)

                                            Missing_Values  \
annual_inc_joint                                     67777   
verification_status_joint                            67780   
hardship_reason                                      72668   
hardship_type                                        72668   
hardship_status                                      72668   
...                                                    ...   
deferral_term                                        72668   
hardship_amount                                      71042   
hardship_payoff_balance_amount                       71042   
hardship_last_payment_amount                         71042   
orig_projected_additional_accrued_interest           71170   

                                            Missing_Percentages  
annual_inc_joint                                      87.840693  
verification_status_joint                             87.844581  
hardship_reason                                       94.

In [80]:
print('Number of Features which has more than 60 peercent of null values: ', len(more_null_values))
more_null_values

Number of Features which has more than 60 peercent of null values:  17


['annual_inc_joint',
 'verification_status_joint',
 'hardship_reason',
 'hardship_type',
 'hardship_status',
 'payment_plan_start_date',
 'dti_joint',
 'mths_since_last_record',
 'mths_since_last_major_derog',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_revol_delinq',
 'revol_bal_joint',
 'deferral_term',
 'hardship_amount',
 'hardship_payoff_balance_amount',
 'hardship_last_payment_amount',
 'orig_projected_additional_accrued_interest']

In [78]:
new_loan_df = loan_data.drop(more_null_values, axis=1)
new_loan_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77159 entries, 0 to 77158
Data columns (total 91 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          77159 non-null  int64  
 1   emp_title                   71297 non-null  object 
 2   application_type            77159 non-null  object 
 3   home_ownership              77159 non-null  object 
 4   loan_status                 77159 non-null  object 
 5   loan_amnt                   77159 non-null  int64  
 6   int_rate                    77159 non-null  object 
 7   term                        77159 non-null  object 
 8   installment                 77159 non-null  float64
 9   grade                       77159 non-null  object 
 10  earliest_cr_line            77159 non-null  object 
 11  issue_d                     77159 non-null  object 
 12  last_pymnt_d                77159 non-null  object 
 13  next_pymnt_d                332